In [1]:
from datetime import datetime

datetime.today()            # 현재 날짜 가져오기

datetime.datetime(2023, 7, 31, 10, 15, 29, 227936)

In [1]:
# !pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import requests
import json
import pandas as pd

In [23]:
# # -> %23, / -> %2F, % -> %25 , & -> %26, + -> %2B, , -> %2C, ! -> %21, ( -> %28, ) -> %29, * -> %2A, ' -> %27, : -> %3A, ; -> %3B, < -> %3C, > -> %3E, ? -> %3F
# url = "http://localhost:8091/epcis/v2/events"
url = "http://localhost:8091/epcis/v2/bizLocations/https%3A%2F%2Fid.oliot.org%2F414%2F880123451006/events"
# url = "http://localhost:8091/epcis/v2/readPoints/https%3A%2F%2Fid.oliot.org%2F414%2F880123451006%2F254%2F20201/events"
response = requests.get(url)

print(response, '\n', response.text)

<Response [200]> 
 {"@context":["https://ref.gs1.org/standards/epcis/2.0.0/epcis-context.jsonld",{"management":"https://example.org/epcis"}],"type":"EPCISQueryDocument","schemaVersion":"2.0","creationDate":"2023-07-27T01:48:37.057Z","epcisBody":{"queryResults":{"queryName":"SimpleEventQuery","resultBody":{"eventList":[{"eventTime":"2023-07-03T20:33:31.116000-06:00","eventTimeZoneOffset":"+09:00","type":"ObjectEvent","action":"OBSERVE","bizStep":"accepting","disposition":"available","epcList":["https://id.oliot.org/8004/8801234562000"],"readPoint":{"id":"https://id.oliot.org/414/880123451006/254/20203"},"bizLocation":{"id":"https://id.oliot.org/414/880123451006"},"recordTime":"2023-07-27T01:46:14.797Z"},{"eventTime":"2023-07-03T20:33:31.116000-06:00","eventTimeZoneOffset":"+09:00","type":"ObjectEvent","action":"OBSERVE","bizStep":"accepting","disposition":"active","epcList":["https://id.oliot.org/8004/8801234562000"],"readPoint":{"id":"https://id.oliot.org/414/880123451006/254/20203"},"

In [2]:
# query 등록
query_json = {
  "name": "objectevent",
  "query": {
    "eventType": [
      "ObjectEvent"
    ]
  }
}

response = requests.post("http://localhost:8091/epcis/v2/queries", json=query_json)
response

# 필요한 데이터별(종류별)로 쿼리를 등록해두고 subscribe해주면?

<Response [201]>

In [30]:
response_1 = requests.get("http://localhost:8091/epcis/v2/queries")  # 등록된 쿼리 확인
response_2 = requests.get("http://localhost:8091/epcis/v2/queries/objectevent/events")  # 특정 쿼리 이벤트 확인
print(response_1.text, '\n',response_2.text)

[{"name":"test01","query":{"eventType":["ObjectEvent"]}}] 
 {"@context":["https://ref.gs1.org/standards/epcis/2.0.0/epcis-context.jsonld",{"management":"https://example.org/epcis"}],"type":"EPCISQueryDocument","schemaVersion":"2.0","creationDate":"2023-07-27T01:51:19.411Z","epcisBody":{"queryResults":{"queryName":"SimpleEventQuery","resultBody":{"eventList":[{"eventTime":"2023-07-03T20:33:31.116000-06:00","eventTimeZoneOffset":"+09:00","type":"ObjectEvent","action":"OBSERVE","bizStep":"accepting","disposition":"available","epcList":["https://id.oliot.org/8004/8801234562000"],"readPoint":{"id":"https://id.oliot.org/414/880123451006/254/20203"},"bizLocation":{"id":"https://id.oliot.org/414/880123451006"},"recordTime":"2023-07-27T01:46:14.797Z"},{"eventTime":"2023-07-03T20:33:31.116000-06:00","eventTimeZoneOffset":"+09:00","type":"ObjectEvent","action":"OBSERVE","bizStep":"accepting","disposition":"active","epcList":["https://id.oliot.org/8004/8801234562000"],"readPoint":{"id":"https://id

In [44]:
# 특정쿼리 구독 등록
# sub_json = {
#   "dest": "https://client.example.com/queryCallback",
#   "signatureToken": "13df38d8275b13f05704629e5f1cf3d45d6132d5",
#   "stream": True
# }

# response = requests.post("http://localhost:8091/epcis/v2/queries/test01/subscriptions", json=sub_json)
# response.text

'{"dest":"https://client.example.com/queryCallback","signatureToken":"13df38d8275b13f05704629e5f1cf3d45d6132d5","stream":true,"createdAt":"2023-07-25T06:26:36.064Z","subscriptionID":"0bbd827c-5261-443f-92e8-f6c28bee0d62","initialRecordTime":"2023-07-25T06:26:36.064Z","reportIfEmpty":false,"minRecordTime":"2023-07-25T06:26:36.064Z","GS1-EPC-Format":"Always_GS1_Digital_Link","queryName":"test01","_id":"64bf6b1c5ef4ba1ca688086d"}'

In [3]:
q_response = requests.get("http://localhost:8091/epcis/v2/queries/objectevent/events")  # 쿼리로 이벤트 가져오기

json_object = json.loads(q_response.text)
event = json_object['epcisBody']['queryResults']['resultBody']['eventList']

df = pd.DataFrame()

for e in event:
    row_data = {
        'eventTime': e['eventTime'],
        'eventTimeZoneOffset': e['eventTimeZoneOffset'],
        'type': e['type'],
        'action': e['action'],
        'bizStep': e['bizStep'],
        'disposition': e.get('disposition'), 
        'epcList': e['epcList'][0],
        'readPoint': e['readPoint']['id'],
        'bizLocation': e['bizLocation']['id'],
        'management:company': e.get('management:company'),
        'sourceList_type': e['sourceList'][0]['type'] if 'sourceList' in e and e['sourceList'] and 'type' in e['sourceList'][0] else None,
        'sourceList_source': e['sourceList'][0]['source'] if 'sourceList' in e and e['sourceList'] and 'source' in e['sourceList'][0] else None,
        'destinationList_type': e['destinationList'][0]['type'] if 'destinationList' in e and e['destinationList'] and 'type' in e['destinationList'][0] else None,
        'destinationList_source': e['destinationList'][0]['destination'] if 'destinationList' in e and e['destinationList'] and 'destination' in e['destinationList'][0] else None,
    }
    df = df.append(row_data, ignore_index=True)

df


,eventTime,eventTimeZoneOffset,type,action,bizStep,disposition,epcList,readPoint,bizLocation,management:company,sourceList_type,sourceList_source,destinationList_type,destinationList_source
0,2023-08-02 17:15:51,+09:00,ObjectEvent,OBSERVE,accepting,active,https://id.oliot.org/8004/8801234562222,https://id.oliot.org/414/880123451006/254/20203,https://id.oliot.org/414/880123451006,None,None,None,None,None
1,2023-08-02 17:15:37,+09:00,ObjectEvent,OBSERVE,accepting,available,https://id.oliot.org/8004/8801234561111,https://id.oliot.org/414/880123451006/254/20203,https://id.oliot.org/414/880123451006,None,None,None,None,None
2,2023-08-02 17:15:12,+09:00,ObjectEvent,OBSERVE,accepting,None,https://id.oliot.org/8004/8801234562222,https://id.oliot.org/414/880123451006/254/30301,https://id.oliot.org/414/880123451006,None,location,https://id.oliot.org/414/880123451006/254/30301,location,https://id.oliot.org/414/880123451006/254/20203
3,2023-08-02 17:14:49,+09:00,ObjectEvent,OBSERVE,accepting,None,https://id.oliot.org/8004/8801234561111,https://id.oliot.org/414/880123451006/254/20201,https://id.oliot.org/414/880123451006,None,location,https://id.oliot.org/414/880123451006/254/20201,location,https://id.oliot.org/414/880123451006/254/20203


In [9]:
q_response = requests.get("http://localhost:8091/epcis/v2/queries/objectevent/events")  # 쿼리로 이벤트 가져오기

json_object = json.loads(q_response.text)
event = json_object['epcisBody']['queryResults']['resultBody']['eventList']

data = []

for e in event:
    row_data = {
        'eventTime': e['eventTime'],
        'eventTimeZoneOffset': e['eventTimeZoneOffset'],
        'type': e['type'],
        'action': e['action'],
        'bizStep': e['bizStep'],
        'disposition': e.get('disposition'), 
        'epcList': e['epcList'][0],
        'readPoint': e['readPoint']['id'],
        'bizLocation': e['bizLocation']['id'],
        'management:company': e.get('management:company'),
        'sourceList_type': e['sourceList'][0]['type'] if 'sourceList' in e and e['sourceList'] and 'type' in e['sourceList'][0] else None,
        'sourceList_source': e['sourceList'][0]['source'] if 'sourceList' in e and e['sourceList'] and 'source' in e['sourceList'][0] else None,
        'destinationList_type': e['destinationList'][0]['type'] if 'destinationList' in e and e['destinationList'] and 'type' in e['destinationList'][0] else None,
        'destinationList_source': e['destinationList'][0]['destination'] if 'destinationList' in e and e['destinationList'] and 'destination' in e['destinationList'][0] else None,
    }
    data.append(row_data)


pd.DataFrame(data)


,eventTime,eventTimeZoneOffset,type,action,bizStep,disposition,epcList,readPoint,bizLocation,management:company,sourceList_type,sourceList_source,destinationList_type,destinationList_source
0,2023-08-02 17:15:51,+09:00,ObjectEvent,OBSERVE,accepting,active,https://id.oliot.org/8004/8801234562222,https://id.oliot.org/414/880123451006/254/20203,https://id.oliot.org/414/880123451006,None,None,None,None,None
1,2023-08-02 17:15:37,+09:00,ObjectEvent,OBSERVE,accepting,available,https://id.oliot.org/8004/8801234561111,https://id.oliot.org/414/880123451006/254/20203,https://id.oliot.org/414/880123451006,None,None,None,None,None
2,2023-08-02 17:15:12,+09:00,ObjectEvent,OBSERVE,accepting,None,https://id.oliot.org/8004/8801234562222,https://id.oliot.org/414/880123451006/254/30301,https://id.oliot.org/414/880123451006,None,location,https://id.oliot.org/414/880123451006/254/30301,location,https://id.oliot.org/414/880123451006/254/20203
3,2023-08-02 17:14:49,+09:00,ObjectEvent,OBSERVE,accepting,None,https://id.oliot.org/8004/8801234561111,https://id.oliot.org/414/880123451006/254/20201,https://id.oliot.org/414/880123451006,None,location,https://id.oliot.org/414/880123451006/254/20201,location,https://id.oliot.org/414/880123451006/254/20203
